In [71]:
import pandas as pd
from sqlalchemy import create_engine

In [72]:
consumption_file = "Resources/alcohol_consumption.csv"
consumption_data_df = pd.read_csv(consumption_file)
consumption_data_df.head()

,Year,Locationabbr,Locationdesc,Class,Topic,Question,Response,Break_Out,Break_Out_Category,Sample_Size,...,Data_Value_Footnote,DataSource,ClassId,TopicId,LocationID,BreakoutID,BreakOutCategoryID,QuestionID,ResponseID,GeoLocation
0,2018,AK,Alaska,Alcohol Consumption,Heavy Drinking,Heavy drinkers (adult men having more than 14 ...,Meet criteria for heavy drinking,"American Indian or Alaskan Native, non-Hispanic",Race/Ethnicity,29,...,NaN,BRFSS,CLASS01,TOPIC30,2,RACE03,CAT4,_RFDRHV6,RESP206,"(64.84507995700051, -147.72205903599973)"
1,2018,AK,Alaska,Alcohol Consumption,Alcohol Consumption,Adults who have had at least one drink of alco...,Yes,35-44,Age Group,219,...,NaN,BRFSS,CLASS01,TOPIC03,2,AGE03,CAT3,DRNKANY5,RESP046,"(64.84507995700051, -147.72205903599973)"
2,2018,AK,Alaska,Alcohol Consumption,Binge Drinking,Binge drinkers (males having five or more drin...,Yes,18-24,Age Group,35,...,NaN,BRFSS,CLASS01,TOPIC07,2,AGE01,CAT3,_RFBING5,RESP046,"(64.84507995700051, -147.72205903599973)"
3,2018,AK,Alaska,Alcohol Consumption,Heavy Drinking,Heavy drinkers (adult men having more than 14 ...,Meet criteria for heavy drinking,Some post-H.S.,Education Attained,49,...,NaN,BRFSS,CLASS01,TOPIC30,2,EDUCA3,CAT5,_RFDRHV6,RESP206,"(64.84507995700051, -147.72205903599973)"
4,2018,AK,Alaska,Alcohol Consumption,Heavy Drinking,Heavy drinkers (adult men having more than 14 ...,Meet criteria for heavy drinking,"$15,000-$24,999",Household Income,15,...,Prevalence estimate not available if the unwei...,BRFSS,CLASS01,TOPIC30,2,INCOME2,CAT6,_RFDRHV6,RESP206,"(64.84507995700051, -147.72205903599973)"


In [73]:
excessive_file = "Resources/excessive_drinking.csv"
excessive_data_df = pd.read_csv(excessive_file)
excessive_data_df.head()

,Edition,Report Type,Measure Name,Age Range 18-44,Age Range 45-64,State Name,Rank,Value,Score,Lower CI,Upper CI,Source,Source Year
0,2019,2019 Annual,Excessive Drinking,Ages 18-44,Ages 45-64,Alabama,3.0,13.8,-1.53,15.1,12.5,"CDC, Behavioral Risk Factor Surveillance System",2018.0
1,2019,2019 Annual,Excessive Drinking,Ages 18-44,Ages 45-64,Alaska,24.0,17.7,-0.17,20.0,15.5,"CDC, Behavioral Risk Factor Surveillance System",2018.0
2,2019,2019 Annual,Excessive Drinking,Ages 18-44,Ages 45-64,Arizona,21.0,17.2,-0.35,18.7,15.8,"CDC, Behavioral Risk Factor Surveillance System",2018.0
3,2019,2019 Annual,Excessive Drinking,Ages 18-44,Ages 45-64,Arkansas,10.0,15.8,-0.83,17.6,14.1,"CDC, Behavioral Risk Factor Surveillance System",2018.0
4,2019,2019 Annual,Excessive Drinking,Ages 18-44,Ages 45-64,California,23.0,17.6,-0.21,18.5,16.7,"CDC, Behavioral Risk Factor Surveillance System",2018.0


In [74]:
consumption_cols = ["Year", "Locationdesc", "Topic", "Class", "Break_Out"]
consumption_transformed = consumption_data_df[consumption_cols].copy()

consumption_transformed = consumption_transformed.rename(columns = {"Locationdesc": "State", "Topic": "Consumption Amount", 
                                                                    "Break_Out": "Age Range"})
consumption_transformed.set_index("State", inplace=True)

consumption_transformed.head()

,Year,Consumption Amount,Class,Age Range
State,,,,
Alaska,2018,Heavy Drinking,Alcohol Consumption,"American Indian or Alaskan Native, non-Hispanic"
Alaska,2018,Alcohol Consumption,Alcohol Consumption,35-44
Alaska,2018,Binge Drinking,Alcohol Consumption,18-24
Alaska,2018,Heavy Drinking,Alcohol Consumption,Some post-H.S.
Alaska,2018,Heavy Drinking,Alcohol Consumption,"$15,000-$24,999"


In [75]:
excessive_cols = ["Source Year", "Measure Name", "State Name", "Rank", "Age Range 18-44", "Age Range 45-64"]
excessive_transformed = excessive_data_df[excessive_cols].copy()

excessive_transformed = excessive_transformed.rename(columns = {"Source Year": "Year", "Measure Name": "Consumption Type", 
                                                                    "State Name": "State", "Age Range 18-44": "Ages", "Age Range 45-64": "Age Range"})
excessive_transformed.set_index("Year", inplace=True)

excessive_transformed.head()

,Consumption Type,State,Rank,Ages,Age Range
Year,,,,,
2018.0,Excessive Drinking,Alabama,3.0,Ages 18-44,Ages 45-64
2018.0,Excessive Drinking,Alaska,24.0,Ages 18-44,Ages 45-64
2018.0,Excessive Drinking,Arizona,21.0,Ages 18-44,Ages 45-64
2018.0,Excessive Drinking,Arkansas,10.0,Ages 18-44,Ages 45-64
2018.0,Excessive Drinking,California,23.0,Ages 18-44,Ages 45-64


In [76]:
connection_string = "postgres:Dog1wood@localhost:5432/consumption_db"
engine = create_engine(f'postgresql://{connection_string}')

In [77]:
engine.table_names()

['consumption', 'excessive_drinking']

In [78]:
consumption_transformed.to_sql(name='consumption', con=engine, if_exists='append', index=False)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "Year" of relation "consumption" does not exist
LINE 1: INSERT INTO consumption ("Year", "Consumption Amount", "Clas...
                                 ^

[SQL: INSERT INTO consumption ("Year", "Consumption Amount", "Class", "Age Range") VALUES (%(Year)s, %(Consumption Amount)s, %(Class)s, %(Age Range)s)]
[parameters: ({'Year': 2018, 'Consumption Amount': 'Heavy Drinking', 'Class': 'Alcohol Consumption', 'Age Range': 'American Indian or Alaskan Native, non-Hispanic'}, {'Year': 2018, 'Consumption Amount': 'Alcohol Consumption', 'Class': 'Alcohol Consumption', 'Age Range': '35-44'}, {'Year': 2018, 'Consumption Amount': 'Binge Drinking', 'Class': 'Alcohol Consumption', 'Age Range': '18-24'}, {'Year': 2018, 'Consumption Amount': 'Heavy Drinking', 'Class': 'Alcohol Consumption', 'Age Range': 'Some post-H.S.'}, {'Year': 2018, 'Consumption Amount': 'Heavy Drinking', 'Class': 'Alcohol Consumption', 'Age Range': '$15,000-$24,999'}, {'Y

In [65]:
excessive_transformed.to_sql(name="excessive_drinking", con=engine, if_exists='append', index=True)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "State" of relation "excessive_drinking" does not exist
LINE 1: INSERT INTO excessive_drinking ("State", "Year", "Consumptio...
                                        ^

[SQL: INSERT INTO excessive_drinking ("State", "Year", "Consumption Type", "Rank", "Ages", "Age Range") VALUES (%(State)s, %(Year)s, %(Consumption Type)s, %(Rank)s, %(Ages)s, %(Age Range)s)]
[parameters: ({'State': 'Alabama', 'Year': 2018.0, 'Consumption Type': 'Excessive Drinking', 'Rank': 3.0, 'Ages': 'Ages 18-44', 'Age Range': 'Ages 45-64'}, {'State': 'Alaska', 'Year': 2018.0, 'Consumption Type': 'Excessive Drinking', 'Rank': 24.0, 'Ages': ' Ages 18-44', 'Age Range': 'Ages 45-64'}, {'State': 'Arizona', 'Year': 2018.0, 'Consumption Type': 'Excessive Drinking', 'Rank': 21.0, 'Ages': 'Ages 18-44', 'Age Range': 'Ages 45-64'}, {'State': 'Arkansas', 'Year': 2018.0, 'Consumption Type': 'Excessive Drinking', 'Rank': 10.0, 'Ages': 'Ages 18-44', 'Age Range': 'Ages 45